In [1]:
import numpy as np
import sys
sys.path.append( '../vqe-in-dft' )
from tqdm import tqdm

import vqe_in_dft 

import scipy as sp
from pyscf import gto, dft, lib, mp, cc, scf, tools, ci, fci

In [2]:
# geometry = [
# ['H', (0.7493682,0.0000000,0.4424329)],
# ['O', (0.0000000,0.0000000,-0.1653507)],
    
# ['H', (-0.7493682,0.0000000,0.4424329)]
# ]

geometry = [
['C', (0.0000, 0.0000, 0.0000)],
['H', (0.5288, 0.1610, 0.9359)],
['H', (0.2051, 0.8240, -0.6786)],
['H', (0.3345, -0.9314, -0.4496)],
['H', (-1.0685, -0.0537, 0.1921)]
]

# geometry = [
# ['H', (1.2473876659, -0.8998737590, 0.6150681570)],
# ['O', (1.2322305822, -0.2731895077, -0.1276123902)],

# ['C', (0.0849758188, 0.5590385475, 0.0510545434)],
# ['H', (0.1506137362, 1.1200249874, 0.9943015309)],
# ['H', (0.1316093068, 1.2841805400, -0.7645223601)],

# ['C', (-1.2129704155, -0.2295285634, -0.0097156258)],
# ['H', (-2.0801425360, 0.4329727646,0.0722817289)],
# ['H', (-1.2655910941, -0.9539857247, 0.8097953440)],
# ['H', (-1.2737541560, -0.7748626513, -0.9540587845)],
# ]

N_active_atoms = 2


low_level_scf_method='RKS'
high_level_scf_method='CCSD'
E_convergence_tol = 1e-6
basis = 'STO-3G' #'augccpvdz' # '6-31g' #'STO-3G'
unit= 'angstrom'
pyscf_print_level=1
memory=8000
charge=0
spin=0
run_fci= True#True
low_level_xc_functional = 'lda, vwn' # 'b3lyp'
high_level_xc_functional = 'b3lyp'

phys_notation = True

# Build the GLOBAL (full) molecule


and run supersystem calculation (cheap method)

https://pyscf.org/_modules/pyscf/gto/basis.html

In [3]:
full_system_mol = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       unit=unit,
                       spin=spin,
                      )
full_system_mol.build()
full_system_mol.atom

[['C', (0.0, 0.0, 0.0)],
 ['H', (0.5288, 0.161, 0.9359)],
 ['H', (0.2051, 0.824, -0.6786)],
 ['H', (0.3345, -0.9314, -0.4496)],
 ['H', (-1.0685, -0.0537, 0.1921)]]

In [4]:
## FCI comparison
HF_scf = scf.RHF(full_system_mol)
HF_scf.verbose=1
HF_scf.max_memory= memory
HF_scf.conv_tol = 1e-6
HF_scf.kernel()

my_fci = fci.FCI(HF_scf).run()
print('E(UHF-FCI) = %.12f' % my_fci.e_tot)

# myci = ci.CISD(HF_scf).run() # this is UCISD
# print('UCISD total energy = ', myci.e_tot)

E(UHF-FCI) = -39.805676967111


In [5]:
# Draw molecule

xyz_string = vqe_in_dft.Get_xyz_string(full_system_mol)
vqe_in_dft.Draw_molecule(xyz_string, width=400, height=400, jupyter_notebook=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
# Run global calculation

full_system_scf = scf.RKS(full_system_mol)
full_system_scf.verbose= pyscf_print_level
full_system_scf.max_memory= memory
full_system_scf.conv_tol = 1e-6
full_system_scf.xc = low_level_xc_functional
full_system_scf.kernel()

-39.61684241519855

# Localise orbitals

### Background

The overlap matrix is:

$$S_{\mu \nu} =  \int d\vec{r}_{1} \phi_{\mu}(1)^{*}\phi_{\nu}(1)$$

- $\phi_{\mu}$ are basis functions (defined in basis set)


The unknown molecular orbitals $\psi_{i}$ are expanded as a linear expansion of the $K$ known basis functions:

$$ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$$


$C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$. The columns of $C$ describe the molecular orbitals!

We can find the total number of electrons $N$ in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  2 \sum_{a}^{N/2} 1$$

- integral gives probablity of finding electron $a$ over all space (must be 1)
- summing over all electrons will give the total number of electrons

The charge density has the following definition:

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg)$$

- re-write using definition of $\psi_{i}=  \sum_{\mu=1}^{K} C_{\mu i} \phi_{\mu}$

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \Bigg( \bigg[ \sum_{\nu}^{K} C_{\nu a}^{*} \phi_{\nu}(\vec{r})^{*} \bigg] \bigg[ \sum_{\mu}^{K} C_{\mu a}\phi_{\mu}(\vec{r}) \bigg] \Bigg)$$

- move things around

$$\rho(\vec{r}) = \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$

- which is 

$$\rho(\vec{r}) = \sum_{\mu, \nu}^{K} P_{\mu \nu} \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$


- $P_{\mu \nu}$ is known as the density matrix and is:

$$P_{\mu \nu} = 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*}$$

Therefore we can also find the total number of electrons in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \int d\vec{r} \phi_{\mu}(\vec{r})  \phi_{\nu}(\vec{r})^{*}$$

- This is simply:

$$N =  \sum_{\nu}^{K} \sum_{\mu}^{K} P_{\mu \nu} S_{\nu \mu}$$

- in an orthognal basis $S$ should be the identity matrix (as it is overlap of matrix of orthogonal orbs), SO WE GET:

$$N =  \sum_{\nu}^{K} \sum_{\mu}^{K} P_{\mu \nu} S_{\nu \mu} = \sum_{\mu}^{K} PS_{\mu \mu} = \mathcal{Tr}(PS)$$

- One can interpret $ PS_{\mu \mu}$ in the above equation as the number of electrons associated with $ \phi_{\mu}$
- This is a **Mulliken population analysis**

# Orbital Localization!

A molecular orbital is usually delocalized, i.e. it has non-negligible amplitude over the whole system rather than only around some atom(s) or bond(s). However, one can choose a unitary rotation 

- When we perform a SCF calculation, one gets an optimized C matrix
    - $C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$
    - The columns of $C$ describe the molecular orbitals!
    - MO i: $ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$
    
    
- These molecular orbitals are usually **delocalized**
    - non-negligible amplitude over the whole system, rather than only around some atom(s) or bond(s)

- But we know in QM that a given basis choice is NOT unique


- We can therefore perform a unitary rotation on molecular orbitals

$$ \psi_{i} U_{rot} =  \Big( \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu} \Big) U_{rot} = \psi_{i}^{new}$$
    
    
The idea is to use a rotation such that the resulting orbitals $\psi_{i}^{new}$ are as spatially localized as possible. 


The Pipek-Mezey (PM) [localization](https://notendur.hi.is/hj/papers/paperPipekmezey8.pdf) **maximizes the population charges on the atoms**:

$$ f (U_{rot}) = \sum_{A}^{N_{atoms}} \Bigg( Z_{A} -  \sum_{\mu \text{ on atom } A} PS_{\mu \mu} \Bigg)$$

# Choose active and environment systems

## METHOD 1 

- Given optimized $C$ coefficient matrix
    - which has been rotated to localize orbitals
    - (used to build localized density matrix)


- **Look through basis functions $\phi_{\mu}$ of the ACTIVE atoms**
    
    
- check the mulliken charge // mulliken population of the orbital
    - if above a certain threshold associate it to active system
    - otherwise put in the environment

## METHOD 2 - SPADE

In [11]:
localization_method= 'PipekMezey' # 'ibo', 'Boys', 'PipekMezey' 'SPADE'
THRESHOLD = 0.95

In [32]:
(C_active, 
 C_envrio, 
 C_all_localized, 
 active_MO_inds,
 enviro_MO_inds) = vqe_in_dft.Localize_orbitals(localization_method, 
                                     full_system_scf, 
                                     N_active_atoms, 
                                     THRESHOLD=THRESHOLD, 
                                     sanity_check=True)



(active_AO^2)/(all_AO^2): [0.9988 0.5689 0.9855 0.5689 0.5689]
threshold for active part: 0.95 

number of active MOs: 2
number of enviro MOs: 3 



In [75]:
dm_loc = 2* C_all_localized@ C_all_localized.conj().T
dm_std = 2* full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]@ full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0].conj().T

print('want these C_loc and C_opt_std to be different:', not np.allclose(C_all_localized,
                                                                         full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]))
print('want dm_std and dm_loc to be the SAME:',np.allclose(dm_std, dm_loc))

want these C_loc and C_opt_std to be different: True
want dm_std and dm_loc to be the SAME: True


In [230]:
dm_loc = 2* C_all_localized@ C_all_localized.conj().T
dm_std = 2* full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]@ full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0].conj().T

print(np.allclose(C_all_localized, full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]))
print(np.allclose(dm_std, dm_loc))

False
True


In [31]:
full_system_scaf.mo_coeff[:,full_system_scf.mo_occ>0].conj().T @ Fock @ full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]

array([[-9.56763009e+00,  8.95591053e-08,  1.42573250e-09,
        -1.39286062e-10,  7.95160706e-10],
       [ 8.95591049e-08, -5.81486991e-01,  5.35071664e-08,
        -9.16699269e-09,  2.77865550e-08],
       [ 1.42573250e-09,  5.35071664e-08, -3.02760345e-01,
        -6.63418266e-10,  1.27899998e-08],
       [-1.39285998e-10, -9.16699267e-09, -6.63418238e-10,
        -3.02737102e-01,  1.02714017e-08],
       [ 7.95160714e-10,  2.77865549e-08,  1.27899997e-08,
         1.02714017e-08, -3.02731084e-01]])

In [19]:
full_system_scf.mo_energy

array([-9.56762769, -0.58148626, -0.30275977, -0.30273646, -0.30273044,
        0.42538041,  0.4253998 ,  0.42544456,  0.4712599 ])

In [97]:
V = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0] @ np.linalg.pinv(C_all_localized)# c
np.allclose(V @C_all_localized,
            full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]) # checking U@ C_loc = C_standard

True

In [125]:
V = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0] @ np.linalg.pinv(C_all_localized)# c
np.allclose(V @C_all_localized,
            full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]) # checking U@ C_loc = C_standard

C_loc_full = (np.linalg.pinv(V) @ full_system_scf.mo_coeff)

print(np.allclose(C_loc_full[:,full_system_scf.mo_occ>0], C_all_localized))
C_loc_full.shape

True


(9, 9)

In [741]:
S_mat = full_system_scf.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)

ortho_std = S_half@ full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]
ortho_loc = S_half@ C_all_localized

# ortho_loc[:,0].dot(ortho_loc[:,2])

U = np.zeros((ortho_std.shape[0], ortho_std.shape[0]))
for MO_ind in range(ortho_std.shape[1]):
    outer = np.outer(ortho_std[:, MO_ind], ortho_loc[:, MO_ind])
    U+=outer

# U = np.zeros((ortho_std.shape[0], ortho_std.shape[0]))
# for MO_ind in range(full_system_scf.mo_coeff.shape[1]):
#     if MO_ind in range(ortho_std.shape[1]):
#         outer = np.outer(ortho_std[:, MO_ind], ortho_loc[:, MO_ind])
#         U+=outer
#     else:
# #         vec = np.eye(ortho_std.shape[0])[:, MO_ind]
# #         outer = np.outer(vec, vec)
# #         U+=outer
#         outer = np.outer(full_system_scf.mo_coeff[:, MO_ind],
#                          full_system_scf.mo_coeff[:, MO_ind])
#         U+=outer
    
print(np.allclose(U @ ortho_loc, ortho_std))
print(np.allclose(U.conj().T@U, U@U.conj().T)) # shows unitary (in given basis)!

C_loc = C_prime @ S_half


ind = 0
print(ortho_loc[:,ind].conj().T @ (U.conj().T @ Fock @ U) @ ortho_loc[:,ind])
print(ortho_std[:,ind].conj().T  @ Fock @ ortho_std[:,ind])

True
True
-10.268228524428004
-10.268228524428077


In [742]:
S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)
np.allclose(S_neg_half@ U@ortho_loc,
            full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0])

True

In [743]:
# U_new = S_neg_half@ U
U_new = S_neg_half@ U  @ S_half
np.allclose(U_new@C_all_localized,
            full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0])


True

In [744]:
print(C_all_localized[:,ind].conj().T @ (U_new.conj().T @ Fock @ U_new) @ C_all_localized[:,ind])
print(full_system_scf.mo_coeff[:,ind].conj().T  @ Fock @ full_system_scf.mo_coeff[:,ind])

-9.567630085619102
-9.567630085619232


In [745]:
np.around(C_all_localized.conj().T @ (U_new.conj().T @ Fock @ U_new) @ C_all_localized, 3)

array([[-9.568,  0.   ,  0.   , -0.   ,  0.   ],
       [ 0.   , -0.581,  0.   , -0.   ,  0.   ],
       [ 0.   ,  0.   , -0.303, -0.   ,  0.   ],
       [-0.   , -0.   , -0.   , -0.303,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   , -0.303]])

In [750]:
np.around(full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0].conj().T @
          Fock @
          full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0], 3)

array([[-9.568,  0.   ,  0.   , -0.   ,  0.   ],
       [ 0.   , -0.581,  0.   , -0.   ,  0.   ],
       [ 0.   ,  0.   , -0.303, -0.   ,  0.   ],
       [-0.   , -0.   , -0.   , -0.303,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   , -0.303]])

In [716]:
np.allclose(S_neg_half@ S_half@ full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0],
                                    full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0])

True

In [727]:
U_new = S_neg_half@ U 
(U_new @ C_all_localized) # full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]   C_all_localized

# full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0] 

array([[ 0.9704987 , -0.13331261, -0.05548858, -0.05547772, -0.05548536],
       [ 0.0863213 ,  0.50830338,  0.01552772,  0.01552151,  0.01553016],
       [-0.04333299,  0.0252617 ,  0.21394727,  0.39552633,  0.04000219],
       [-0.0233079 ,  0.01358778, -0.05696776, -0.06398999,  0.47028546],
       [-0.01156471,  0.00673481,  0.41685834, -0.2495368 ,  0.00595607],
       [-0.03188138,  0.15969803,  0.39410042, -0.02153516,  0.08626857],
       [-0.01832905,  0.15179206, -0.21899966,  0.1611507 ,  0.31360852],
       [ 0.00734208,  0.13681849, -0.04361495,  0.24475696, -0.33015266],
       [ 0.03321635,  0.12173357, -0.10859483, -0.3614485 , -0.04681425]])

In [731]:

U_new = S_neg_half.conj().T@U @ S_neg_half

(U_new @ C_all_localized)#.shape
# full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]#.shape

array([[ 0.96659774, -0.06116092, -0.09537741, -0.09536116, -0.09537226],
       [ 0.11182129,  0.41962661,  0.01468029,  0.01467952,  0.01468689],
       [-0.07824129,  0.02715603,  0.1711742 ,  0.3217222 ,  0.02695641],
       [-0.04208428,  0.01460824, -0.0505674 , -0.0563966 ,  0.3865748 ],
       [-0.02088133,  0.00723837,  0.34395956, -0.20854827,  0.00328658],
       [-0.04854599,  0.13481045,  0.32434298, -0.02025976,  0.06912485],
       [-0.02407572,  0.12631783, -0.18203328,  0.13314572,  0.25954786],
       [ 0.02227607,  0.11022057, -0.03294942,  0.20614598, -0.27051627],
       [ 0.06899464,  0.09400404, -0.08311355, -0.29275384, -0.03188908]])

In [677]:
ortho_test = V@full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]

ortho_test[:,0].dot(ortho_test[:,4])

-1.1500731860431858e-05

In [670]:
V = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp(), -0.5)

F_ortho = V.conj().T @  full_system_scf.get_fock(dm=dm_std) @ V
epsilon_std, C_prime_std = np.linalg.eigh(F_ortho)
print(np.allclose(epsilon_std, full_system_scf.mo_energy))
C_std = V @ C_prime_std

print(np.around(C_std.conj().T @ full_system_scf.get_fock(dm=dm_std) @ C_std, 2))

# F_bas = U.conj().T @  full_system_scf.get_fock(dm=dm_std) @ U # new basis
# F_prime = V.conj().T @ F_bas @ V # make ortho

F_ortho = V.conj().T @  full_system_scf.get_fock(dm=dm_std) @ V # make ortho
F_prime = U.conj().T @ F_ortho @ U # new basis ortho
epsilon_loc, C_prime_loc = np.linalg.eigh(F_prime)
print(np.allclose(epsilon_loc[full_system_scf.mo_occ>0], full_system_scf.mo_energy[full_system_scf.mo_occ>0]))
C_local = V @ U @ C_prime_loc 

# # transofrm C_prime back to AO basis
# C_std = V @ C_prime_std
# C_local = V @ U @ C_prime_loc 

True
[[-9.57  0.   -0.   -0.    0.    0.    0.    0.    0.  ]
 [ 0.   -0.58  0.    0.   -0.   -0.    0.   -0.    0.  ]
 [-0.    0.   -0.3   0.   -0.   -0.   -0.   -0.   -0.  ]
 [ 0.    0.    0.   -0.3   0.    0.   -0.    0.   -0.  ]
 [ 0.   -0.   -0.    0.   -0.3  -0.   -0.   -0.   -0.  ]
 [ 0.   -0.   -0.    0.   -0.    0.43 -0.    0.   -0.  ]
 [ 0.    0.   -0.   -0.   -0.   -0.    0.43  0.    0.  ]
 [ 0.   -0.   -0.    0.    0.    0.    0.    0.43 -0.  ]
 [ 0.    0.   -0.    0.   -0.   -0.    0.   -0.    0.47]]
True


In [661]:
print(np.around(C_local.conj().T @ U.conj().T @ full_system_scf.get_fock(dm=dm_std) @ U @ C_local, 1))

[[-10.3  -1.9  -0.5   0.5   0.5   0.6  -0.1  -0.2  -0. ]
 [ -1.9  -1.1  -0.    0.    0.    0.2  -0.1  -0.1  -0. ]
 [ -0.5  -0.   -0.3   0.    0.    0.1  -0.   -0.   -0. ]
 [  0.5   0.    0.   -0.3  -0.   -0.1   0.    0.   -0. ]
 [  0.5   0.    0.   -0.   -0.3  -0.1   0.    0.   -0. ]
 [  0.6   0.2   0.1  -0.1  -0.1  -0.1   0.    0.    0. ]
 [ -0.1  -0.1  -0.    0.    0.    0.   -0.   -0.   -0. ]
 [ -0.2  -0.1  -0.    0.    0.    0.   -0.   -0.   -0. ]
 [ -0.   -0.   -0.   -0.   -0.    0.   -0.   -0.   -0. ]]


In [628]:
# np.around(full_system_scf.mo_coeff.conj().T @ Fock @ full_system_scf.mo_coeff, 1)
# np.around(C_all_localized.conj().T @ (U.conj().T @ Fock @ U) @ C_all_localized, 1)

F_bas = V.conj().T @  full_system_scf.get_fock(dm=dm_std) @ V # make ortho
F_prime = U.conj().T @ F_bas @ U # new basis ortho
epsilon_loc, C_prime_loc = np.linalg.eigh(F_prime)


print(np.allclose(epsilon_loc[full_system_scf.mo_occ>0], full_system_scf.mo_energy[full_system_scf.mo_occ>0]))


C_local = V @ C_prime_loc 

np.around(C_local.conj().T @ (U.conj().T @ Fock @ U) @ C_local, 1)

True


array([[-10.3,  -1.9,  -0.5,   0.5,   0.5,   0.6,  -0.1,  -0.2,  -0. ],
       [ -1.9,  -1.1,  -0. ,   0. ,   0. ,   0.2,  -0.1,  -0.1,  -0. ],
       [ -0.5,  -0. ,  -0.3,   0. ,   0. ,   0.1,  -0. ,  -0. ,  -0. ],
       [  0.5,   0. ,   0. ,  -0.3,  -0. ,  -0.1,   0. ,   0. ,  -0. ],
       [  0.5,   0. ,   0. ,  -0. ,  -0.3,  -0.1,   0. ,   0. ,  -0. ],
       [  0.6,   0.2,   0.1,  -0.1,  -0.1,  -0.1,   0. ,   0. ,   0. ],
       [ -0.1,  -0.1,  -0. ,   0. ,   0. ,   0. ,  -0. ,  -0. ,  -0. ],
       [ -0.2,  -0.1,  -0. ,   0. ,   0. ,   0. ,  -0. ,  -0. ,  -0. ],
       [ -0. ,  -0. ,  -0. ,  -0. ,  -0. ,   0. ,  -0. ,  -0. ,  -0. ]])

In [611]:
V = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp(), -0.5)

# F_prime = U.conj().T @ np.linalg.pinv(V) @ full_system_scf.get_fock(dm=dm_loc) @ V @ U
F_ortho = V.conj().T @  full_system_scf.get_fock(dm=dm_std) @ V
epsilon_std, C_prime_std = np.linalg.eigh(F_ortho)
print(np.allclose(epsilon_std, full_system_scf.mo_energy))

# F_bas = U.conj().T @  full_system_scf.get_fock(dm=dm_std) @ U # new basis
# F_prime = V.conj().T @ F_bas @ V # make ortho

F_bas = V.conj().T @  full_system_scf.get_fock(dm=dm_std) @ V # make ortho
F_prime = U.conj().T @ F_bas @ U # new basis ortho
epsilon_loc, C_prime_loc = np.linalg.eigh(F_prime)
print(np.allclose(epsilon_loc, full_system_scf.mo_energy))
print(np.allclose(epsilon_loc[full_system_scf.mo_occ>0], full_system_scf.mo_energy[full_system_scf.mo_occ>0]))

# transofrm C_prime back to AO basis
C_std = V @ C_prime_std
C_local = V @ U @ C_prime_loc 

True
False
True


In [597]:
np.around(C_std.conj().T @ full_system_scf.get_fock(dm=dm_std) @ C_std, 2)

array([[-9.57,  0.  , -0.  , -0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.58,  0.  ,  0.  , -0.  , -0.  ,  0.  , -0.  ,  0.  ],
       [-0.  ,  0.  , -0.3 ,  0.  , -0.  , -0.  , -0.  , -0.  , -0.  ],
       [ 0.  ,  0.  ,  0.  , -0.3 ,  0.  ,  0.  , -0.  ,  0.  , -0.  ],
       [ 0.  , -0.  , -0.  ,  0.  , -0.3 , -0.  , -0.  , -0.  , -0.  ],
       [ 0.  , -0.  , -0.  ,  0.  , -0.  ,  0.43, -0.  ,  0.  , -0.  ],
       [ 0.  ,  0.  , -0.  , -0.  , -0.  , -0.  ,  0.43,  0.  ,  0.  ],
       [ 0.  , -0.  , -0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.43, -0.  ],
       [ 0.  ,  0.  , -0.  ,  0.  , -0.  , -0.  ,  0.  , -0.  ,  0.47]])

In [610]:
np.around(C_local.conj().T @ F_bas @ C_local, 1)

array([[-8.4, -1.9,  0.8,  0.4,  0.2,  0.1, -0.1, -0.6, -1. ],
       [-1.9, -0.5,  0.2,  0.1,  0.1,  0. , -0.1, -0.2, -0.3],
       [ 0.8,  0.2, -0.1, -0. , -0. , -0. ,  0. ,  0. ,  0.2],
       [ 0.4,  0.1, -0. , -0.1, -0. , -0. , -0. ,  0.1,  0.1],
       [ 0.2,  0.1, -0. , -0. , -0.1, -0. ,  0. ,  0. ,  0. ],
       [ 0.1,  0. , -0. , -0. , -0. , -0.1,  0. ,  0. ,  0. ],
       [-0.1, -0.1,  0. , -0. ,  0. ,  0. , -0.1, -0. , -0. ],
       [-0.6, -0.2,  0. ,  0.1,  0. ,  0. , -0. , -0.1, -0.1],
       [-1. , -0.3,  0.2,  0.1,  0. ,  0. , -0. , -0.1, -0.2]])

In [547]:
# V = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp(), -0.5)

F_ortho = U.conj().T @ full_system_scf.get_fock(dm=dm_std) @ U
# F_ortho = V.conj().T @  F_new_basis @ V

epsilon_std, C_prime_std = np.linalg.eigh(F_ortho)
epsilon_std

array([-10.58751774,  -1.13654259,  -0.44207985,  -0.44202685,
        -0.43929045,   1.15678835,   1.15694903,   1.15707904,
         2.11280491])

In [554]:
# np.around(C_prime_loc.conj().T @ F_prime @ C_prime_loc, 2)

array([[-9.57,  0.  , -0.  , -0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.58,  0.  ,  0.  , -0.  , -0.  ,  0.  , -0.  ,  0.  ],
       [-0.  ,  0.  , -0.3 ,  0.  , -0.  , -0.  , -0.  , -0.  , -0.  ],
       [ 0.  ,  0.  ,  0.  , -0.3 ,  0.  ,  0.  , -0.  ,  0.  , -0.  ],
       [ 0.  , -0.  , -0.  ,  0.  , -0.3 , -0.  , -0.  , -0.  , -0.  ],
       [ 0.  , -0.  , -0.  ,  0.  , -0.  ,  0.43, -0.  ,  0.  , -0.  ],
       [ 0.  ,  0.  , -0.  , -0.  , -0.  , -0.  ,  0.43,  0.  ,  0.  ],
       [ 0.  , -0.  , -0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.43, -0.  ],
       [ 0.  ,  0.  , -0.  ,  0.  , -0.  , -0.  ,  0.  , -0.  ,  0.47]])

array([[-10.3,  -1.9,  -0.5,   0.5,   0.5,   0.6,  -0.1,  -0.2,  -0. ],
       [ -1.9,  -1.1,  -0. ,   0. ,   0. ,   0.2,  -0.1,  -0.1,  -0. ],
       [ -0.5,  -0. ,  -0.3,   0. ,   0. ,   0.1,  -0. ,  -0. ,  -0. ],
       [  0.5,   0. ,   0. ,  -0.3,  -0. ,  -0.1,   0. ,   0. ,  -0. ],
       [  0.5,   0. ,   0. ,  -0. ,  -0.3,  -0.1,   0. ,   0. ,  -0. ],
       [  0.6,   0.2,   0.1,  -0.1,  -0.1,  -0.1,   0. ,   0. ,   0. ],
       [ -0.1,  -0.1,  -0. ,   0. ,   0. ,   0. ,  -0. ,  -0. ,  -0. ],
       [ -0.2,  -0.1,  -0. ,   0. ,   0. ,   0. ,  -0. ,  -0. ,  -0. ],
       [ -0. ,  -0. ,  -0. ,  -0. ,  -0. ,   0. ,  -0. ,  -0. ,  -0. ]])

In [452]:
epsilon_std, C_prime_std  = full_system_scf.eig(full_system_scf.get_fock(dm=dm_std), 
                                                s= full_system_scf.get_ovlp())
C_std = V @ C_prime_std
np.allclose(C_std, full_system_scf.mo_coeff)

False

In [444]:
# full_system_scf.mo_coeff

In [441]:
V@C_prime_std

array([[-9.88460251e-01, -2.27996606e-01,  7.82275752e-06,
         2.34740163e-06,  6.09205471e-06,  5.63545368e-05,
         6.80964598e-05, -4.97034392e-05, -2.59546809e-01],
       [-5.54027723e-02,  6.32586529e-01, -2.43723962e-05,
        -1.15782641e-05, -2.11715734e-05, -3.47157700e-04,
        -4.03422684e-04,  3.18093085e-04,  1.62043905e+00],
       [ 7.68413963e-07, -1.50662001e-05, -2.88874842e-01,
        -5.08033663e-01, -7.98214576e-02, -7.52450189e-02,
        -9.06732001e-01,  6.07291068e-01, -4.04506573e-04],
       [ 5.30175591e-07, -1.01326704e-05,  5.26454217e-02,
         6.18550358e-02, -5.84230122e-01, -1.09127081e+00,
         5.99821115e-02, -4.56666563e-02, -2.37379999e-04],
       [ 4.44612533e-07, -6.91385463e-06, -5.11551835e-01,
         2.93252113e-01, -1.50519755e-02, -4.54905627e-03,
         6.08960983e-01,  9.08753365e-01, -4.15132159e-05],
       [ 1.13464504e-02,  1.79022402e-01, -4.89723814e-01,
         1.23858707e-02, -1.18206757e-01,  1.977426

In [383]:
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)
C_loc = S_half @ ortho_loc 
C_loc

(9, 5)

In [372]:
V = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp(), -0.5)

# F_prime = U.conj().T @ np.linalg.pinv(V) @ full_system_scf.get_fock(dm=dm_loc) @ V @ U
F_ortho = V.conj().T @  full_system_scf.get_fock(dm=dm_std) @ V
F_prime = U.conj().T @ F_ortho @ U

epsilon, C_prime = np.linalg.eigh(F_prime)

# transofrm C_prime back to AO basis
C = U @ C_prime

np.allclose(np.abs(full_system_scf.mo_coeff), np.abs(C))

False

In [361]:
U = np.zeros((ortho_std.shape[0], ortho_std.shape[0]))
for MO_ind in range(ortho_std.shape[1]):
    outer = np.outer(ortho_loc[:, MO_ind], ortho_loc[:, MO_ind])
    U+=outer
    
np.allclose(U @ ortho_loc, ortho_loc)
np.allclose(U.conj().T@U, U@U.conj().T)
np.allclose(U @ ortho_std, ortho_loc)

False

In [242]:
from functools import reduce

zero =np.array([[1],[0]])
one = np.array([[0],[1]])

first = [one for i in range(full_system_scf.mol.nelectron//2)]
second = [zero for i in range(full_system_scf.mol.nelectron//2, F_rot.shape[1])]

state = reduce(np.kron, [*first, *second])

In [250]:
F_MO = full_system_scf.mo_coeff.conj().T @ Fock @ full_system_scf.mo_coeff


print(full_system_scf.mo_energy)
print(np.diag(F_MO))

[-9.56762769 -0.58148626 -0.30275977 -0.30273646 -0.30273044  0.42538041
  0.4253998   0.42544456  0.4712599 ]
[-9.56763009 -0.58148699 -0.30276035 -0.3027371  -0.30273108  0.4253797
  0.42539908  0.42544391  0.47125932]


In [241]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
F_rot = C_loc_full.conj().T @ (U.conj().T@ Fock @U) @ C_loc_full

# F_MO = full_system_scf.mo_coeff.conj().T @ Fock @ full_system_scf.mo_coeff
print(np.around(F_rot, 1))

state.conj().T @ F_rot @ state

[[-0.5 -0.1  0.1  0.  -0.1  0.  -0.   0.  -0.1]
 [-0.1 -0.2  0.4 -0.1 -0.3  0.   0.   0.  -0.2]
 [ 0.1  0.4 -1.8  0.5  1.6 -0.2 -0.1 -0.2  0.4]
 [ 0.  -0.1  0.5 -0.4 -0.4  0.   0.   0.  -0.1]
 [-0.1 -0.3  1.6 -0.4 -1.8  0.2  0.1  0.2 -0.3]
 [ 0.   0.  -0.2  0.   0.2 -0.  -0.  -0.   0. ]
 [-0.   0.  -0.1  0.   0.1 -0.  -0.  -0.   0. ]
 [ 0.   0.  -0.2  0.   0.2 -0.  -0.  -0.   0. ]
 [-0.1 -0.2  0.4 -0.1 -0.3  0.   0.   0.  -0.2]]


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 9 is different from 512)

In [168]:
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)
ortho_loc = S_half@ C_all_localized

print(ortho_loc[:,0].dot(ortho_loc[:,3]))

# U, singular_values, rotation_matrix = np.linalg.svd(ortho_loc, full_matrices=True)


# act_orbitals = ortho_loc @ rotation_matrix.T[:, active_MO_inds]
# env_orbitals = ortho_loc @ rotation_matrix.T[:, enviro_MO_inds]

# C_matrix_all_localized_orbitals =  C_all_localized @ rotation_matrix.T

# C_matrix_all_localized_orbitals

5.981543385602528e-16


In [220]:
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)
ortho_loc = S_half@ C_all_localized

act_orbitals = ortho_loc[:, active_MO_inds]
env_orbitals = ortho_loc[:, enviro_MO_inds]

S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)
C_active =  S_neg_half @ act_orbitals
C_enviro =  S_neg_half @ env_orbitals


dm_active = 2*C_active @ C_active.conj().T
dm_enviro = 2*C_enviro @ C_enviro.conj().T
dm_loc = 2*C_all_localized @ C_all_localized.conj().T


print(np.allclose(C_active, C_all_localized[:, active_MO_inds]))
print(np.allclose(C_enviro, C_all_localized[:, enviro_MO_inds]))

np.allclose(dm_loc, dm_active+dm_enviro)

True
True


True

True
True


In [204]:
projector_ortho = np.zeros_like(full_system_scf.get_hcore())
for mo_ind in range(ortho_loc.shape[1]):
    psi_mo = ortho_loc[:, mo_ind]
    if mo_ind in active_MO_inds:
        projector_ortho+= np.outer(psi_mo,psi_mo)

np.allclose(projector_ortho@ortho_loc[:, 2],
            ortho_loc[:, 2])

True

In [215]:
Proj_std = S_neg_half @ projector_ortho

Proj_std@C_active[:, 0] 


array([ 0.99586356,  0.00956646, -0.01229789, -0.00374187, -0.02176068,
       -0.04566708, -0.01687342, -0.0168775 , -0.01687389])

In [195]:
projector = np.zeros_like(full_system_scf.get_hcore())
for mo_ind in range(C_active.shape[1]):
    psi_mo = C_active[:, mo_ind]
    projector+= np.outer(psi_mo,psi_mo)

np.allclose(projector@C_active[:, 0],
            C_active[:, 0])

False

In [193]:
projector = np.zeros_like(full_system_scf.get_hcore())
for mo_ind in range(ortho_loc.shape[1]):
    psi_mo = ortho_loc[:, mo_ind]
    if mo_ind in active_MO_inds:
        projector+= np.outer(psi_mo,psi_mo)

np.allclose(projector@ortho_loc[:, 2],
            ortho_loc[:, 2])

True

In [229]:
# test1 = full_system_scf.mo_coeff.conj().T @ full_system_scf.get_fock() @ full_system_scf.mo_coeff
# np.around(test1,2)

# test2 =   full_system_scf.mo_coeff  @ full_system_scf.mo_coeff.conj().T @ full_system_scf.get_fock()
# np.around(test2,2)

True

In [ ]:
from pyscf import lo
mo_occ = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]
iaos = lo.iao.iao(full_system_scf.mol, mo_occ)
# Orthogonalize IAO
iaos = lo.vec_lowdin(iaos, full_system_scf.get_ovlp())
C_all_localized = lo.ibo.ibo(full_system_scf.mol, mo_occ, locmethod='PipekMezey', iaos=iaos)#.kernel()

C_all_localized

In [ ]:
from pyscf.lo.boys import Boys
from pyscf.lo import PipekMezey

boy_Scf = PipekMezey(full_system_scf.mol,  full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0])
boy_Scf.kernel()
C_all_localized = boy_Scf.mo_coeff
C_all_localized

In [ ]:
np.allclose(C_all_localized, full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0])

In [86]:
full_system_scf.mo_energy

array([-9.56762769, -0.58148626, -0.30275977, -0.30273646, -0.30273044,
        0.42538041,  0.4253998 ,  0.42544456,  0.4712599 ])

In [88]:
S_neg = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp(), -1)

# F_prime = U.conj().T @ np.linalg.pinv(V) @ full_system_scf.get_fock(dm=dm_loc) @ V @ U
Sneg_F = S_neg @  full_system_scf.get_fock(dm=dm_std) 
(C_all_localized[:,2].conj().T @ Sneg_F@ C_all_localized[:,2])


-0.22153123282052678

In [68]:
U = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp(), -0.5)

# F_prime = U.conj().T @ np.linalg.pinv(V) @ full_system_scf.get_fock(dm=dm_loc) @ V @ U
F_prime = U.conj().T @  full_system_scf.get_fock(dm=dm_std) @ U


epsilon, C_prime = np.linalg.eigh(F_prime)

# transofrm C_prime back to AO basis
C = U @ C_prime

np.allclose(np.abs(full_system_scf.mo_coeff), np.abs(C))

False

In [ ]:
# S_ovlp = full_system_scf.get_ovlp()
# AO_slice_matrix = full_system_scf.mol.aoslice_by_atom()
# ao_active_inds = np.arange(AO_slice_matrix[0,2], AO_slice_matrix[N_active_atoms-1,3])

# dm_test = 2* C_all_localized @ C_all_localized.conj().T
# # dm_test = 2* full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0] @ full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0].conj().T

# DS = dm_test@S_ovlp

# # ao_active_inds
# diagonal_active = DS[ao_active_inds, ao_active_inds]

# diagonal_active


In [ ]:
# S_ovlp = full_system_scf.get_ovlp()
# AO_slice_matrix = full_system_scf.mol.aoslice_by_atom()
# ao_active_inds = np.arange(AO_slice_matrix[0,2], AO_slice_matrix[N_active_atoms-1,3])

# S_half = sp.linalg.fractional_matrix_power(S_ovlp , 0.5)
# orthogonal_orbitals = (S_half@C_all_localized) # Get rows (the active AO) of orthogonal orbs 

# print(orthogonal_orbitals[:,2].dot(orthogonal_orbitals[:,3]))

# MO_AO_overlap = S_ovlp@orthogonal_orbitals
# MO_active_AO_overlap = np.einsum('ij->j', MO_AO_overlap[ao_active_inds]) 

# print('\noverlap:', MO_active_AO_overlap)
# print(f'threshold for active part: {THRESHOLD} \n')

# np.where(MO_active_AO_overlap>THRESHOLD)[0]

In [ ]:
# # Draw active orbitals

# list_active_orbitals = vqe_in_dft.Draw_cube_orbital(full_system_scf,
#                              xyz_string,
#                              C_all_localized, 
#                              active_MO_inds,
#                              width=400, 
#                              height=400, 
#                              jupyter_notebook=True)

# list_enviro_orbitals = vqe_in_dft.Draw_cube_orbital(full_system_scf,
#                              xyz_string,
#                              C_all_localized, 
#                              enviro_MO_inds,
#                              width=400, 
#                              height=400, 
#                              jupyter_notebook=True)

In [ ]:
# list_active_orbitals[0]

In [ ]:
# list_enviro_orbitals[0]

In [ ]:
# AO_slice_matrix = full_system_scf.mol.aoslice_by_atom()
# ao_active_inds = np.arange(AO_slice_matrix[0,2], AO_slice_matrix[N_active_atoms-1,3])
# ao_enviro_inds = np.arange(AO_slice_matrix[N_active_atoms,2], AO_slice_matrix[-1,-1])

# U, S, V = np.linalg.svd(C_all_localized)

# Projector_out_env = np.eye(C_all_localized.shape[0])
# Projector_out_env[ao_enviro_inds, ao_enviro_inds]= np.zeros((len(ao_enviro_inds),len(ao_enviro_inds)))
# Projector_out_env

In [ ]:
dm_active, dm_enviro = vqe_in_dft.Get_active_and_envrio_dm(
                                                full_system_scf,
                                                C_active, 
                                                C_envrio, 
                                                C_all_localized, 
                                                sanity_check=True)

In [ ]:
# use active density
E_act, J_act, K_act, e_xc_act, v_xc_act = vqe_in_dft.Get_energy_and_matrices_from_dm( full_system_scf, 
                                                                             dm_active, # <- ACTIVE
                                                                             check_E_with_pyscf=True)

In [ ]:
# use enviro density
E_env, J_env, K_env, e_xc_env, v_xc_env = vqe_in_dft.Get_energy_and_matrices_from_dm(
                                                                            full_system_scf,
                                                                             dm_enviro, # <- ENVIRO
                                                                             check_E_with_pyscf=True)

In [ ]:
# cross terms!
two_e_cross = vqe_in_dft.Get_cross_terms(full_system_scf, 
                                         dm_active, 
                                         dm_enviro, 
                                         J_env, 
                                         J_act, 
                                         e_xc_act,
                                         e_xc_env)

In [ ]:
print(f'E_active: {E_act}')
print(f'E_enviro: {E_env}')
print(f'E_cross: {two_e_cross}')

# 4. Define V_embed

$$h^{A_{\text{act}} \text{ in } B_{\text{env}}} = h^{\text{core}} + v^{\text{embed}}$$

$$v^{\text{embed}} = g[\gamma^{A_{\text{act}}} + \gamma^{B_{\text{env}}} ] - g[\gamma^{A_{\text{act}}}] + P_{\text{projector}}$$

In [ ]:
projector_method = 'huzinaga'
# projector_method = 'mu_shfit'

V_embed  = vqe_in_dft.Get_embedded_potential_operator(projector_method, 
                                full_system_scf, 
                                dm_active, 
                                dm_enviro, 
                                check_Hcore_is_correct=True, 
                                mu_shift_val=1e6,
                                check_Vemb=True)

In [ ]:
# projector_method = 'huzinaga'
# # projector_method = 'mu_shfit'

# if projector_method == 'huzinaga':
#     Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm=dm_active + dm_enviro)
#     F_gammaB_S = Fock @ dm_enviro @ S_mat
#     projector = -0.5 * (F_gammaB_S + F_gammaB_S.T)
# elif projector_method == 'mu_shfit':
#     mu = 1e6
#     projector = mu * (S_mat @ dm_enviro  @ S_mat)
# else:
#     raise ValueError(f'Unknown projection method {projector_method}')



In [ ]:
full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]

In [ ]:
IND=2
print(full_system_scf.mo_coeff[:,IND].conj().T @ full_system_scf.get_fock() @ full_system_scf.mo_coeff[:,IND])
print(full_system_scf.mo_energy[IND])

In [ ]:
IND=4
print(C_all_localized[:,IND].conj().T @ full_system_scf.get_fock() @ C_all_localized[:,IND])
print(full_system_scf.mo_energy[IND])

In [ ]:
U = full_system_scf.mo_coeff @ np.linalg.pinv(C_all_localized)# c
np.allclose(U @C_all_localized,
            full_system_scf.mo_coeff) # checking U@ C_loc = C_standard

In [ ]:
U1, singular_values1, rotation_matrix = np.linalg.svd(full_system_scf.get_fock(dm = dm_active + dm_enviro)
                                                      , full_matrices=True)

U2, singular_values2, rotation_matrix2 = np.linalg.svd(full_system_scf.get_fock()
                                                      , full_matrices=True)

singular_values1

In [ ]:
U1, singular_values1, rotation_matrix = np.linalg.svd(C_all_localized[:,full_system_scf.mo_occ>0], full_matrices=True)
# U2, singular_values2, V2 = np.linalg.svd(full_system_scf.mo_coeff, full_matrices=True)

C_active = C_all_localized[:,full_system_scf.mo_occ>0] @ rotation_matrix.T[:, active_MO_inds]
C_envrio = C_all_localized[:,full_system_scf.mo_occ>0] @ rotation_matrix.T[:, enviro_MO_inds]


C_matrix_all_localized_orbitals = C_all_localized[:,full_system_scf.mo_occ>0] @ rotation_matrix.T


In [ ]:
dm_localised_full_system = 2* C_matrix_all_localized_orbitals@ C_matrix_all_localized_orbitals.conj().T
dm_active =  2 * C_active @ C_active.T
dm_enviro =  2 * C_envrio @ C_envrio.T
    
bool_density_flag = np.allclose(dm_localised_full_system, dm_active + dm_enviro)
bool_density_flag

In [ ]:
V_dag, singular_values, V = np.linalg.svd(U, full_matrices=True)

print(C_all_localized[:,IND].conj().T @ (V@full_system_scf.get_fock() @ V_dag)/2 @ C_all_localized[:,IND])
print(full_system_scf.mo_energy[IND])

In [ ]:
IND=2
print(C_all_localized[:,IND].conj().T @ (U.conj().T@full_system_scf.get_fock() @ U) @ C_all_localized[:,IND])
print(full_system_scf.mo_energy[IND])

In [ ]:
eig_vals, eig_vecs = np.linalg.eigh((U.conj().T@full_system_scf.get_fock() @ U))

eig_vals2, eig_vecs2 = np.linalg.eigh(full_system_scf.get_fock())

print(eig_vals)
print(eig_vals2)

In [ ]:
dm_loc = 2 * C_all_localized[:,full_system_scf.mo_occ>0] @ C_all_localized[:,full_system_scf.mo_occ>0].conj().T

C_all_localized[:,IND].conj().T @ full_system_scf.get_fock() @ C_all_localized[:,IND]

In [ ]:
C_opt_std = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]
dm_std = 2*C_opt_std @ C_opt_std.conj().T
np.allclose(dm_std, dm_active+dm_enviro)

In [ ]:
dm_active + dm_ac
np.allclose(full_system_scf.get_veff(dm=dm_active + dm_enviro), 
            full_system_scf.get_veff())

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
eig_vals, eig_vecs = np.linalg.eigh(Fock)

eig_vals

In [ ]:
full_system_scf.mo_energy

In [ ]:
fasdfds

In [ ]:
Fock@full_system_scf.mo

In [ ]:
(U.conj().T@Fock@U).shape

In [ ]:
U = C_all_localized @ np.linalg.pinv(full_system_scf.mo_coeff)# c

np.allclose(U @ full_system_scf.mo_coeff,
C_all_localized)

In [ ]:
# ind = 2
# print(full_system_scf.mo_coeff[:,ind].conj().T @ full_system_scf.get_fock() @ full_system_scf.mo_coeff[:,ind])
# print(full_system_scf.mo_energy[ind]) 

# F_mo = C_all_localized.conj().T @ (U.conj().T@ full_system_scf.get_fock() @U) @ C_all_localized
# np.around(F_mo, 1)

In [ ]:
C_all_localized.conj().T @ full_system_scf.get_fock() @ C_all_localized

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
F_rot = C_all_localized.conj().T @ (U.conj().T@ Fock @U) @ C_all_localized

# F_MO = full_system_scf.mo_coeff.conj().T @ Fock @ full_system_scf.mo_coeff
print(np.around(F_rot, 1))

In [ ]:
AO_slice_matrix = full_system_scf.mol.aoslice_by_atom()
ao_active_inds = np.arange(AO_slice_matrix[0,2], AO_slice_matrix[N_active_atoms-1,3])

ao_ENViro_inds = np.arange(AO_slice_matrix[N_active_atoms,2], AO_slice_matrix[-1,-1])

print(ao_active_inds)
print(ao_ENViro_inds)

In [ ]:
ao_ENViro_inds

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_enviro)
S_mat = full_system_scf.get_ovlp()

P = np.zeros_like(Fock)
# for i in ao_ENViro_inds:
for mu in ao_ENViro_inds:
    if mu in ao_ENViro_inds:
        for nu in ao_ENViro_inds:
            if nu in ao_ENViro_inds:
                P[mu, nu]+= dm_enviro[mu, nu] * (Fock[mu, mu]*S_mat[nu, nu] + S_mat[mu, mu]*Fock[nu, nu])
    
P.shape

In [ ]:
# orbital_ind=4
# psi_MO = EMBEDDED_full_system_scf.mo_coeff[:, orbital_ind]
# E_MO =EMBEDDED_full_system_scf.mo_energy[orbital_ind]

# Fock = EMBEDDED_full_system_scf.get_fock()
# np.allclose(psi_MO.conj().T @Fock@psi_MO, E_MO)

# Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
# projector = np.zeros_like(Fock)
# for orbital_ind in range(full_system_scf.mo_coeff.shape[1]):
#     psi_MO = EMBEDDED_full_system_scf.mo_coeff[:, orbital_ind]
#     if orbital_ind in active_MO_inds:
#         outer = np.outer(psi_MO, psi_MO)
#         projector-= Fock@outer + outer@Fock
#     else:
#        # inactive 
#         continue

Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
projector = np.zeros_like(Fock)
for orbital_ind in range(C_all_localized.shape[1]):
    psi_MO = C_all_localized[:, orbital_ind]
    if orbital_ind in active_MO_inds:
        outer = np.outer(psi_MO, psi_MO)
        projector-= Fock@outer + outer@Fock
    else:
       # inactive 
        continue

In [ ]:
# Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
# S_mat = full_system_scf.get_ovlp()

# projector = np.zeros_like(Fock)
# for mu in range(Fock.shape[0]):
#     for nu in range(Fock.shape[1]):
#         projector[mu, nu] = -0.5* (dm_enviro[mu, nu]) * (Fock[mu,mu]*S_mat[nu, nu] + S_mat[mu,mu]*Fock[nu, nu])


In [ ]:
# PP = -0.5 * dm_enviro @ (Fock@S_mat+ S_mat@Fock)
# # np.allclose(projector, PP)

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
projector = np.zeros_like(Fock)
for orbital_ind in range(C_all_localized.shape[1]):
    psi_MO = C_all_localized[:, orbital_ind]
    if orbital_ind in active_MO_inds:
        outer = np.outer(psi_MO, psi_MO)
        projector = outer
    else:
       # inactive 
        continue

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
projector = np.zeros_like(Fock)
for orbital_ind in range(C_all_localized.shape[1]):
    psi_MO = C_all_localized[:, orbital_ind]
    if orbital_ind in enviro_MO_inds: # <---- ENV
        outer = np.outer(psi_MO, psi_MO)
        projector = Fock@outer + Fock@outer
    else:
       # inactive 
        continue

In [ ]:
C_all_localized[:, orbital_ind]

In [ ]:
orbital_ind=1

mo_E = full_system_scf.mo_energy[orbital_ind]

( (Fock) @ C_all_localized[:, orbital_ind])/ (C_all_localized[:, orbital_ind].conj().T@ Fock@ C_all_localized[:, orbital_ind])


In [ ]:
# F_MO =  C_all_localized.conj().T @ Fock @ C_all_localized
F_MO = full_system_scf.mo_coeff.conj().T @ Fock @ full_system_scf.mo_coeff
print(np.around(F_MO, 2))


In [ ]:
Proj_MO = np.zeros_like(F_MO)
# Proj_MO[ao_active_inds, ao_active_inds]=1
Proj_MO[ao_ENViro_inds, ao_ENViro_inds]= -1* F_MO[ao_ENViro_inds, ao_ENViro_inds]


Proj_AO = (full_system_scf.mo_coeff.conj().T@ Proj_MO 
           @ np.linalg.pinv(full_system_scf.mo_coeff))

# Proj_AO

In [ ]:
MO_ind=6
print(np.eye(Proj_MO.shape[0])[:,MO_ind] @ Proj_MO @ np.eye(Proj_MO.shape[0])[:,MO_ind])
print(np.eye(Proj_MO.shape[0])[:,MO_ind] @ F_MO @ np.eye(Proj_MO.shape[0])[:,MO_ind])
print()
print(np.eye(Proj_MO.shape[0])[:,MO_ind] @ (F_MO+Proj_MO) @ np.eye(Proj_MO.shape[0])[:,MO_ind])

In [ ]:
C_inv = np.linalg.pinv(full_system_scf.mo_coeff)
projected_fock_AO = (C_inv.conj().T@ (Proj_MO+F_MO)@ C_inv)

In [ ]:
ind = 6
full_system_scf.mo_coeff[:,ind].conj().T @ projected_fock_AO @ full_system_scf.mo_coeff[:,ind]

In [ ]:
full_system_scf.mo_coeff[:,ind].conj().T @ Fock @ full_system_scf.mo_coeff[:,ind]

In [ ]:
full_system_scf.mo_coeff[:,1]

In [ ]:
Proj_AO@full_system_scf.mo_coeff[:,1]

In [ ]:
np.allclose(np.linalg.pinv(full_system_scf.mo_coeff.conj().T)@ F_MO @ np.linalg.pinv(full_system_scf.mo_coeff),
           Fock)

In [ ]:
full_system_scf.mo_coeff[:, orbital_ind]

In [ ]:
orbital_ind=3

print(np.isclose(F_MO[orbital_ind,orbital_ind], full_system_scf.mo_energy[orbital_ind]))

np.allclose(F_MO @ full_system_scf.mo_coeff[:, orbital_ind],
            full_system_scf.mo_coeff[:, orbital_ind]/full_system_scf.mo_energy[orbital_ind])


In [ ]:
orbital_ind=3
projector @ full_system_scf.mo_coeff[:, orbital_ind]

In [ ]:
C_all_localized[:, orbital_ind]

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
projector = np.zeros_like(Fock)
for orbital_ind in range(C_all_localized.shape[1]):
    if orbital_ind in enviro_MO_inds:
        outer = np.outer(psi_MO, psi_MO)
        projector-= Fock@outer + outer@Fock
    else:
       # inactive 
        continue

In [ ]:
orbital_ind=0
C_all_localized[:, orbital_ind].conj().T @ (Fock+projector)@ C_all_localized[:, orbital_ind]

In [ ]:
active_MO_inds

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)

FP = (Fock + projector)

orbital_ind=2
psi_MO = EMBEDDED_full_system_scf.mo_coeff[:, orbital_ind]
psi_MO.conj().T @FP@psi_MO


In [ ]:
# psi_MO = EMBEDDED_occ_orbs[:, 1]

# P_B = 1e6* S_mat@ dm_enviro @ S_mat
# P_B = -0.5*(Fock@dm_enviro@S_mat + S_mat@dm_enviro@Fock)

for orbital_ind in range(EMBEDDED_full_system_scf.mo_coeff.shape[1]):
    psi_MO = EMBEDDED_full_system_scf.mo_coeff[:, orbital_ind]
    print(f'orb ind {orbital_ind}')
#     print(f'<MO| Vemb |MO > : {2*(psi_MO @ V_embed @ psi_MO)} \n')
    print(f'<MO| Pb |MO > : {2*(psi_MO @ P @ psi_MO)} \n')

In [ ]:
# Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
# S_mat = full_system_scf.get_ovlp()

# Fock_A = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active)

In [ ]:
# F_A = Fock[:ao_active_inds[-1], :ao_active_inds[-1]]

# gamma_tot = 2 * C_all_localized @ C_all_localized.conj().T

# # gamma_act = gamma_tot[:ao_active_inds[-1]+1, :ao_active_inds[-1]+1]
# # gamma_env = gamma_tot[ao_ENViro_inds[0]:, ao_ENViro_inds[0]:]
# gamma_act = gamma_tot.copy()
# gamma_act[ao_ENViro_inds[0]:, ao_ENViro_inds[0]:] = np.zeros((len(ao_ENViro_inds), len(ao_ENViro_inds)))

# gamma_env = gamma_tot.copy()
# gamma_env[:ao_active_inds[-1]+1, :ao_active_inds[-1]+1] = np.zeros((len(ao_active_inds), len(ao_active_inds)))

# dm_active = gamma_act
# gamma_env = gamma_env
# V_embed  = vqe_in_dft.Geat_embedded_potential_operator(projector_method, 
#                                 full_system_scf, 
#                                 dm_active, 
#                                 dm_enviro, 
#                                 check_Hcore_is_correct=True, 
#                                 mu_shift_val=1e6,
#                                 check_Vemb=True)

In [ ]:
Fock = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active+ dm_enviro)
S_mat = full_system_scf.get_ovlp()

Fock_A = full_system_scf.get_hcore() + full_system_scf.get_veff(dm = dm_active)

In [ ]:
# # psi_MO = EMBEDDED_occ_orbs[:, 1]

# P_B = 1e6* S_mat@ dm_enviro @ S_mat
# # P_B = -0.5*(Fock@dm_enviro@S_mat + S_mat@dm_enviro@Fock)

# for orbital_ind in range(EMBEDDED_full_system_scf.mo_coeff.shape[1]):
#     psi_MO = EMBEDDED_full_system_scf.mo_coeff[:, orbital_ind]
#     print(f'orb ind {orbital_ind}')
# #     print(f'<MO| Vemb |MO > : {2*(psi_MO @ V_embed @ psi_MO)} \n')
#     print(f'<MO| Pb |MO > : {2*(psi_MO @ P_B @ psi_MO)} \n')

In [ ]:
# np.around(Fock_A@P_B - P_B@Fock_A, 5)

# 5. Run RKS DFT of full system with $V_{emb}$ to get $\gamma_{emb}^{\text{active}}$ 

In [ ]:
full_system_mol_EMBEDDED = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf = scf.RKS(full_system_mol_EMBEDDED)
EMBEDDED_full_system_scf.verbose=1
EMBEDDED_full_system_scf.max_memory= memory
EMBEDDED_full_system_scf.conv_tol = 1e-6
EMBEDDED_full_system_scf.xc = low_level_xc_functional

h_core = EMBEDDED_full_system_scf.get_hcore()

# overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf.get_hcore = lambda *args: V_embed + h_core

E_emb = EMBEDDED_full_system_scf.kernel()

print(f'embedded Energy: {E_emb}')

In [ ]:
EMBEDDED_full_system_scf.conv_check

In [ ]:
EMBEDDED_occ_orbs = EMBEDDED_full_system_scf.mo_coeff[:,EMBEDDED_full_system_scf.mo_occ>0]

# optimized embedded denisty matrix
density_emb = 2 * EMBEDDED_occ_orbs @ EMBEDDED_occ_orbs.conj().T

## check number of electrons makes sense:
electron_check = np.isclose(np.trace(density_emb@full_system_scf.get_ovlp()), 2*len(active_MO_inds))

print(f'number of e- in gamma_embedded is correct: {electron_check}')

In [ ]:
# calculate embedding correction term

dm_correction = np.einsum('ij, ij', V_embed, density_emb-dm_active)
WF_correction = np.einsum('ij, ij', V_embed, dm_active)

print(f'RKS correction: {dm_correction}')
print(f'WF correction: {WF_correction}')

In [ ]:
## PsiEmbed Way

# J_emb, K_emb =EMBEDDED_full_system_scf.get_jk(dm=density_emb) 

# # note this uses the STANDARD H_core
# matrix_dot = lambda A, B: np.einsum('ij,ij', A, B)
# e_act_emb = matrix_dot(density_emb, h_core + 0.5 * J_emb - 0.25 * K_emb)
# e_act_emb

## MY way
e_act_emb = full_system_scf.energy_elec(dm=density_emb,
                                        vhf= full_system_scf.get_veff(dm=density_emb),
                                       h1e = h_core)[0]
e_act_emb

In [ ]:
e_mf_emb = e_act_emb + E_env + two_e_cross + full_system_scf.energy_nuc() + dm_correction
e_mf_emb # <-- energy from embedded DFT calc

In [ ]:
full_system_scf.e_tot

In [ ]:
print(f'global DFT calculation == seperated calculation: {np.isclose(e_mf_emb, full_system_scf.e_tot)}')
# expected as same functional used!

# 6. High level DFT calc!

In [ ]:
full_system_mol_HIGH_LEVEL_DFT = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )

full_system_mol_HIGH_LEVEL_DFT.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!
full_system_mol_HIGH_LEVEL_DFT.build()

full_system_scf_HIGH_LEVEL = scf.RKS(full_system_mol_HIGH_LEVEL_DFT)
full_system_scf_HIGH_LEVEL.verbose=1
full_system_scf_HIGH_LEVEL.max_memory= memory
full_system_scf_HIGH_LEVEL.conv_tol = 1e-6
full_system_scf_HIGH_LEVEL.xc = high_level_xc_functional # <-- BETTER functional!


# full_system_scf_HIGH_LEVEL.kernel() < --- do NOT RUN THIS

In [ ]:
# run energy calc using high level functional (note density matrix is embedded A)

e_act_emb_HIGH_LVL = full_system_scf_HIGH_LEVEL.energy_elec(dm=density_emb)
e_act_emb_HIGH_LVL 

In [ ]:
E_high_lvl_DFT = e_act_emb_HIGH_LVL[0] + E_env + two_e_cross + full_system_scf.energy_nuc() + dm_correction
E_high_lvl_DFT # <-- energy from embedded DFT calc

In [ ]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

# 7. High level WF calc (classical run) !

In [ ]:
full_system_mol_EMBEDDED_HF = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED_HF.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED_HF.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf_HF = scf.RHF(full_system_mol_EMBEDDED) # <---- Hartree Fock Calc!
EMBEDDED_full_system_scf_HF.verbose=1
EMBEDDED_full_system_scf_HF.max_memory= memory
EMBEDDED_full_system_scf_HF.conv_tol = 1e-6

h_core = EMBEDDED_full_system_scf_HF.get_hcore()

# overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: V_embed + h_core

# EMBEDDED_full_system_scf_HF.kernel() # <------ do NOT RUN!

In [ ]:
# overwrite orbs with RKS embedded orbs!
EMBEDDED_full_system_scf_HF.mo_coeff = EMBEDDED_full_system_scf.mo_coeff 
EMBEDDED_full_system_scf_HF.mo_occ = EMBEDDED_full_system_scf.mo_occ 
EMBEDDED_full_system_scf_HF.mo_energy = EMBEDDED_full_system_scf.mo_energy

print(EMBEDDED_full_system_scf_HF.energy_elec(dm=density_emb)[0])
print(EMBEDDED_full_system_scf_HF.energy_tot(dm=density_emb))
print()
print(EMBEDDED_full_system_scf_HF.energy_elec()[0])
print(EMBEDDED_full_system_scf_HF.energy_tot())

In [ ]:
# EMBEDDED_full_system_scf_HF.kernel()
# FCI_embedded = fci.FCI(EMBEDDED_full_system_scf_HF)
# E_FCI,FCI_vec = FCI_embedded.kernel()
# # from pyscf.ci.cisd import  to_fcivec
# # cisdWFN   = to_fcivec(C_FCI, len(active_MO_inds), 2*len(active_MO_inds))

# RDM1 = FCI_embedded.make_rdm1(FCI_vec,
#                        norb=EMBEDDED_full_system_scf_HF.mo_coeff.shape[1],
#                        nelec=2*len(active_MO_inds))

# V,S, U = np.linalg.svd(RDM1)
# print(np.isclose(2*len(active_MO_inds), np.sum(S)))
# print(S)

# print('')
# print('ignore: ', S[])

In [ ]:
## CCSD calculation

embedded_cc_obj = cc.CCSD(EMBEDDED_full_system_scf_HF)

embedded_cc_obj.frozen = [i for i in range(EMBEDDED_full_system_scf_HF.mol.nao - len(enviro_MO_inds),
                                           EMBEDDED_full_system_scf_HF.mol.nao)
                         ]
# embedded_cc_obj.frozen  = enviro_MO_inds.tolist()

e_cc, t1, t2 = embedded_cc_obj.kernel()

CC_flag_check = np.isclose(EMBEDDED_full_system_scf_HF.energy_tot(dm=density_emb),
                          embedded_cc_obj.e_hf)


print(f'E_corr: {e_cc}')
print(f'\nCC hartree fock energy matches HF embedded calc: {CC_flag_check}!')
embedded_cc_obj.e_hf

In [ ]:
print('orbital energies:')
print(EMBEDDED_full_system_scf_HF.mo_energy)

In [ ]:
print(embedded_cc_obj.frozen)

In [ ]:
WF_correction  = np.einsum('ij, ij', V_embed, dm_active) # note different definition

E_WF = embedded_cc_obj.e_hf +e_cc  + E_env + two_e_cross - WF_correction
E_WF 

In [ ]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))

In [ ]:
drive_obj = vqe_in_dft.embeddeding_SCF_driver(geometry,
                 N_active_atoms,
                 projector_method,
                 cheap_global_SCF_method='RKS', 
                 cheap_global_DFT_xc= 'lda, vwn',
                 expensive_global_DFT_xc = 'b3lyp',
                 cheap_WF_method = 'RHF',
                 expensive_WF_method = 'CCSD',
                 E_convergence_tol = 1e-6,
                 basis = basis,
                 unit= 'angstrom',
                 pyscf_print_level=1,
                 memory=8000,
                 charge=0,
                 spin=0,
                 run_fci=False,
                 run_cisd=False,
                 mu_value=1e6,
                 physists_notation= False)#phys_notation)

E_DFT_low, E_DFT_high, E_Classical, E_QC, H_ferm = drive_obj.run_experiment(localization_method, 
                                                                      orbtial_loc_threshold=THRESHOLD,
                                                                   jupyter_notebook=True)

In [ ]:
# np.binary_repr(52).count('1')  

In [ ]:
print('LOW level DFT in DFT error:', np.abs(E_DFT_low-my_fci.e_tot))
print('High level DFT in DFT error:', np.abs(E_DFT_high-my_fci.e_tot))
print('WF in DFT error:', np.abs(E_Classical-my_fci.e_tot))
print('VQE in DFT error:', np.abs(E_QC-my_fci.e_tot))

In [ ]:
print(e_mf_emb)
print(E_high_lvl_DFT)
print(E_WF)

# QC part

$$H_{fermionic} = h_{nuc} + \sum_{p, q} h_{pq} a_{p}^{\dagger} a_{q} + \frac{1}{2} \sum_{p, q ,r ,s} h_{pqrs} a_{p}^{\dagger} a_{q}^{\dagger} a_{r} a_{s}$$

where:

- $$h_{pq} = \int d{\vec{x}} \phi_{p}^{*}({\vec{x}}) \bigg( - \frac{\nabla^{2}_{{\vec{r}}}}{2} - \sum_{I} \frac{Z_{I}}{|{\vec{r}} - {\vec{R}_{I}} |} \bigg) \phi_{q}({\vec{x}})$$


- $$h_{pqrs} = \int d{\vec{x}}_{1} d{\vec{x}}_{2} \frac{\phi_{p}^{*}({\vec{x}}_{1}) \phi_{q}^{*}({\vec{x}}_{2}) \phi_{s}({\vec{x}}_{1}) \phi_{r}({\vec{x}}_{2})}{|{\vec{r}_{1}} - {\vec{r}}_{2}|}$$ 


- $$h_{nuc} = \frac{1}{2} \sum_{I \neq J} \frac{Z_{I} Z_{J}}{| {\vec{R}}_{I} - {\vec{R}}_{J}|}$$

note project means we don't want last set of MO orbs filled!

In [ ]:
# EMBEDDED_full_system_scf_HF.mo_coeff[:]

In [ ]:
N_enviroment_MOs = len(enviro_MO_inds) 

one_body_integrals, two_body_integrals = vqe_in_dft.Get_embedded_one_and_two_body_integrals_MO_basis(EMBEDDED_full_system_scf_HF,
                                                                                            N_enviroment_MOs,
                                                                                            physists_notation=phys_notation)

In [ ]:
print(one_body_integrals.shape)
print(two_body_integrals.shape)

print(active_MO_inds)
print(enviro_MO_inds)

In [ ]:
one_body_terms, two_body_terms = vqe_in_dft.Get_SpinOrbs_from_Spatial(one_body_integrals,
                                                                   two_body_integrals,
                                                                   physists_notation=phys_notation,
                                                                   EQ_Tolerance=1e-8)

In [ ]:
Nuclear_energy =  full_system_scf.energy_nuc()

H_fermionic = vqe_in_dft.Get_fermionic_H(one_body_terms, 
                                     two_body_terms, 
                                     Nuclear_energy,
                                     core_constant=0, 
                                     physists_notation=phys_notation,
                                        jupyter_notebook=True)

In [ ]:
len(list(H_fermionic))

In [ ]:
from openfermion.linalg import get_sparse_operator
H_sparse = get_sparse_operator(H_fermionic)

In [ ]:
eigvals_EMBED, eigvecs_EMBED = sp.sparse.linalg.eigsh(H_sparse, which='SA', k=1)
eigvals_EMBED

In [ ]:
WF_correction  = np.einsum('ij, ij', V_embed, dm_active) # note different definition

E_VQE = eigvals_EMBED[0]  + E_env + two_e_cross - WF_correction
E_VQE 

In [ ]:
N_electrons_expected = 2*len(active_MO_inds)
N_electrons_Q_state = np.binary_repr(np.where(np.abs(eigvecs_EMBED)>1e-2)[0][0]).count('1') 

print(f'expect {N_electrons_expected} electrons')
print(f'quantum state has {N_electrons_Q_state} electrons \n')

print(f'number of electrons correct: {N_electrons_expected == N_electrons_Q_state}')

In [ ]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))


print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))
print('VQE error:', np.abs(E_VQE-my_fci.e_tot))

In [ ]:
where=np.where(np.around(eigvecs_EMBED, 4)>0)[0]
print(where)

np.binary_repr(where[0]).count('1') 